Tahapan Deploy Django Prodject

In [ ]:
1.	Pastikan Anda telah mengakses VPS Anda melalui SSH menggunakan nama pengguna root

    - Untuk mengakses robot terminal pada VPS anda terlebih dahulu anda mengetahui SSH Keygen Komputer anda
      Berikut totorial yang dapat untuk memperoleh SSH Keygen
      https://www.youtube.com/watch?v=N9YrHvyT1rs&t=26s
    - Setelah memperoleh SSH Keygen, pada halaman VPS  pilih menu Settings => SSH Key => add SSH Key
    - Setelah ditampilkan form add SSH Key anda dapat mmenginputkan nama sesuai keinginan ande dan SSH Key sesuai dengan
      SSH Keygen yang sebelumnya telah anda dapatkan
    - Selanjutnya pada laman VPS anda, Buka menu Overview => SSH access => Copy terminal contoh
      Terminal:      ssh root@31.220.107.145
    - Buka terminal bawaan anda, disini saya menggunakan terminal GIT karena terminal VPS menggunakan OS Ubuntu
    - Pastekan ssh yang sebelumnya anda copy ke dalam terminal GIT dan tekan enter. Secara otomatis terminal GIT anda akan
      terkoneksi dengan terminal VPS anda

In [ ]:
2. apt update

In [ ]:
3. apt upgrade

In [ ]:
4.cek versi
  apache2 -v
  python --version
  apache2ctl -M
  pip --version
  SQLite is Included with Python
  python -c "import sqlite3; print(sqlite3.sqlite_version)"

In [ ]:
5. Jika belum terinstall

    - apt install apache2
    - apt install python
    - apt install libapache2-mod-wsgi-py3
    - apt install python3-pip

In [ ]:
6. Pastikan anda telah menginstal gunicorn, WhiteNoise dan membuat file gunicorn_config.py agar kita dapat menjalankan website kita 
   secara life
    - install gunicorn
      pip install gunicorn
    - isian file gunicorn_config.py
    
import os
from whitenoise import WhiteNoise
from Mywebsite.wsgi import application

# Assuming your virtual environment is in /var/www/Skripsi/ven
activate_this = '/var/www/Skripsi/ven/bin/activate_this.py'
exec(open(activate_this).read(), dict(__file__=activate_this))

application = WhiteNoise(application, root=os.path.join(os.path.dirname(__file__), '/static/'))
STATIC_ROOT = os.path.join(os.path.dirname(__file__), '/static/')
#application = WhiteNoise(application, root=STATIC_ROOT)

bind = "0.0.0.0:8000"
workers = 4
timeout = 240
#forwarded_allow_ips = '*'
#secure_scheme_headers = {'X-FORWARDED-PROTO': 'http'}

chdir = '/var/www/Skripsi'  # Path to your Django project

# Static files
static_path = '/var/www/Skripsi/static'  # Path to your static files
static_url = '/static/'

# Django WSGI module
#module = 'Mywebsite.wsgi:application'


In [ ]:
7. Ada beberapa hal yang perlu di setting terlebih dahulu didalam file settings.py

    - ALLOWED_HOSTS = ['127.0.0.1', 'mmradikal.site', '31.220.107.145'] tambahkan alamat IP dan domain anda
    - CSRF_TRUSTED_ORIGINS = ['https://mmradikal.site', 'https://31.220.107.145'] membiarkan CSRF terhadap domain dan IP anda
    - Mengubah
      STATICFILES_DIRS = [
            BASE_DIR / 'static',
      ]
        
        Menjadi
      import os
      STATIC_ROOT = os.path.join(BASE_DIR, '/static/')

In [ ]:
8. Sebelum melakukan push proyek django anda kedalam Github pastikan terdapat file requirements.txt

   pip freeze > requirements.txt

In [ ]:
9. Melakukan push proyek django anda kedalam Github
10. Pada terminal VPS, buka directory cd /var/www/ sebagai tempat penyimpanan proyek django anda kemudian clone proyek django 
   yang telah di push kedalam Github

In [ ]:
11. Membuat env baru dengan menjalankan
    python3 -m venv myenv

In [ ]:
12. Mengumpulkan file static dengan menjalankan 
    python manage.py collectstatic

In [ ]:
13. Membuat file json untuk mengamankan SECRET_KEY

    Contoh
    nano /etc/config.json yang berisikan SECRET_KEY
    
      GNU nano 6.2                                                                  /etc/config.json
    {
            "SECRET_KEY" : "django-insecure-+n$gjvo=#4b04*21-x$j!)7pgd^0pf5kvauwbp%+m4b9u9*@n="
    }

In [ ]:
14. Configurasikan config.json dengan proyek django anda dengan mengubah pengaturan pada settings.py menjadi


from pathlib import Path
import json

with open('/etc/config.json') as config_files:
    config = json.load(config_files)
# Build paths inside the project like this: BASE_DIR / 'subdir'.
BASE_DIR = Path(__file__).resolve().parent.parent


# Quick-start development settings - unsuitable for production
# See https://docs.djangoproject.com/en/4.2/howto/deployment/checklist/

# SECURITY WARNING: keep the secret key used in production secret!
SECRET_KEY = config['SECRET_KEY']

In [ ]:
15. Konfigurasikan server apache2 anda dengan proyek django
    - buka cd /etc/apache2/sites-available/
    - kopikan konfigurasi defult 000-default.conf kedalam konfigurasi baru sebagai konfigurasi proyek django anda
    
    cp defult 000-default.conf django_project.conf
    
    - Ubah setingan django_project.conf menjadi
    
<VirtualHost *:80>
        # The ServerName directive sets the request scheme, hostname and port that
        # the server uses to identify itself. This is used when creating
        # redirection URLs. In the context of virtual hosts, the ServerName
        # specifies what hostname must appear in the request's Host: header to
        # match this virtual host. For the default virtual host (this file) this
        # value is not decisive as it is used as a last resort host regardless.
        # However, you must set it for any further virtual host explicitly.
        #ServerName www.example.com

        ServerAdmin webmaster@localhost
        ServerName mmradikal.site
        DocumentRoot /var/www/Skripsi

        ProxyPass /static/ !
        ProxyPass / http://127.0.0.1:8000/
        ProxyPassReverse / http://127.0.0.1:8000/

#        RequestHeader set X-Forwarded-Proto "http"
        # Available loglevels: trace8, ..., trace1, debug, info, notice, warn,
        # error, crit, alert, emerg.
        # It is also possible to configure the loglevel for particular
        # modules, e.g.
        #LogLevel info ssl:warn

        <Directory /var/www/Skripsi>
                AllowOverride All
        </Directory>


        Alias /static /var/www/Skripsi/static
       <Directory /var/www/Skripsi/static>
                Require all granted
        </Directory>

        <Directory /var/www/Skripsi/Mywebsite>
                <Files wsgi.py>
                        Require all granted
                </Files>
        </Directory>

        WSGIDaemonProcess Skripsi python-home=/var/www/Skripsi/ven python-path=/var/www/Skripsi
        WSGIProcessGroup Skripsi
        WSGIScriptAlias / /var/www/Skripsi/Mywebsite/wsgi.py

        ErrorLog ${APACHE_LOG_DIR}/error.log
        CustomLog ${APACHE_LOG_DIR}/access.log combined

        # For most configuration files from conf-available/, which are
        # enabled or disabled at a global level, it is possible to
        # include a line for only one particular virtual host. For example the
        # following line enables the CGI configuration for this host only
        # after it has been globally disabled with "a2disconf".
        #Include conf-available/serve-cgi-bin.conf

RewriteEngine on
RewriteCond %{SERVER_NAME} =mmradikal.site
RewriteRule ^ https://%{SERVER_NAME}%{REQUEST_URI} [END,NE,R=permanent]
</VirtualHost>


In [ ]:
16. Konfigurasi Database dengan settings.py
    - Membuat Database didalam VPS
        * Masuk kedalam mysql anda
          mysql -u root -p
        * jika anda lupa pasword anda dapat membuat user baru pada mysql
        
          https://www.youtube.com/watch?v=CB9rsOYv71Q
                
        * Setelah membuat user baru dan masuk kedalam mysql tahapan selanjutnya membuat database baru
          CREATE DATABASE nama_database;
    - Melakukan seting Database pada sttings.py
      
        DATABASES = {
            'default': {
                'ENGINE': 'django.db.backends.mysql',
                'NAME': 'DataPenjualan',  # Ganti dengan nama database yang Anda buat
                'USER': 'ikal123',  # Ganti dengan nama pengguna MySQL Anda
                'PASSWORD': '@ZXCVqwer123',  # Ganti dengan kata sandi MySQL Anda
                'HOST': 'localhost',
                'PORT': '3306',
                'OPTIONS': {
                    'init_command': "SET sql_mode='STRICT_TRANS_TABLES'",
                    'charset': 'utf8mb4',
                },
            }
        }
17. migrasi Database
18. Membuat Superuser

In [ ]:
19. Memastikan port 8000 mendapatkan izin 
    - ufw status
    - ufw allow 8000

In [ ]:
20. Membuat konfigurasi gunicorn agar tidak dijalankan manual
    - membuat file configurasi baru
      nano /etc/systemd/system/gunicorn.service
    - seting konfigurasi

[Unit]
Description=Gunicorn service for My Project
After=network.target

[Service]
User=root
Group=root
WorkingDirectory=/var/www/Skripsi
ExecStart=/var/www/Skripsi/ven/bin/gunicorn Mywebsite.wsgi:application -c /var/www/Skripsi/gunicorn_config.py --log-level debug

[Install]
WantedBy=multi-user.target

In [ ]:
21. Menjalankan dan memastikan gunicorn berjalan dengan normal
    - systemctl daemon-reload
    - systemctl restart gunicorn
    - journalctl -u gunicorn -xe
    - systemctl status gunicorn
    - systemctl start gunicorn

In [ ]:
22. Memastikan kembali server apache2 berjalan
    - systemctl restart apache2
    - journalctl -u apache2 -xe
    - systemctl status apache2
    - systemctl start apache2

cek error
tail -f /var/log/apache2/error.log

In [ ]:
23. Mengubah http menjadi https
    https://youtu.be/dYdv6pkCufk?si=rnHGx1M6oMZfVXO6